In [ ]:
import numpy as np
import sklearn
import time as tm
from scipy.linalg import khatri_rao
from sklearn.svm import LinearSVC

In [ ]:
def my_fit( X_train, y_train ):

	features = my_map(X_train)
	labels = y_train

	model = LinearSVC()
	model.fit(features, labels)
	w = model.coef_.flatten()
	b = model.intercept_[0]

	return w,b

In [ ]:
# def reduce_dimensionality_kronecker_product(X):
#     num_columns = X.shape[1]
#     result = []

#     for i in range(num_columns):
#         for j in range(i, num_columns):
#             product = X[:, i] * X[:, j]
#             result.append(product)

#     return np.array(result).T

# def my_map( X ):
#   D = 2 * X - 1
#   E = np.flip(D, axis = 1)
#   F = np.cumprod(E, axis = 1)
#   G = np.flip(F, axis = 1)
#   H = reduce_dimensionality_kronecker_product(G)

#   return H

# ###################
# def my_map2( X ):
#   G = np.flip(np.cumprod(np.flip(2 * X - 1, axis = 1), axis = 1), axis = 1)

#   num_columns = G.shape[1]
#   result = []

#   for i in range(num_columns):
#       for j in range(i, num_columns):
#           product = G[:, i] * G[:, j]
#           result.append(product)

#   return np.array(result).T

# tic = tm.perf_counter()
# features = my_map2(Z_trn)
# toc = tm.perf_counter()
# t_map = toc - tic
# print(t_map)
# print(features.shape)


# ############################################
# def my_map4( X ):
#   G = np.flip(np.cumprod(np.flip(2 * X - 1, axis = 1), axis = 1), axis = 1)
#   H = khatri_rao(G.T, G.T)

#   return H.T

# tic = tm.perf_counter()
# features = my_map3(Z_trn)
# toc = tm.perf_counter()
# t_map = toc - tic
# print(t_map)
# print(features.shape)

In [ ]:
# BEST TIME:
def my_map( X ):
  X = np.flip(np.cumprod(np.flip(2 * X - 1, axis = 1), axis = 1), axis = 1)
  X = np.hstack((X, np.ones((X.shape[0], 1))))
  num_columns = X.shape[1]
  feat = np.empty((X.shape[0], num_columns * (num_columns + 1) // 2))
  idx = 0

  for i in range(num_columns):
      feat[:, idx:idx+num_columns-i] = X[:, i:] * X[:, i][:, np.newaxis]
      idx += num_columns - i

  return feat

# EVALUATION SCRIPT:

In [ ]:
Z_trn = np.loadtxt( "secret_train.dat" )
Z_tst = np.loadtxt( "secret_test.dat" )

# The variable n_trials is set to 5, indicating that the
# subsequent operations will be repeated 5 times in a loop.
n_trials = 5

# These variables will be used to accumulate values across multiple trials.
d_size = 0
t_train = 0
t_map = 0
acc = 0

In [ ]:
# features = my_map(Z_trn[:, :-1])
# features.shape

(40000, 561)

In [ ]:
# A for loop is initiated to repeat the following operations
# for each trial, where t takes values from 0 to 4 (5 trials).

for t in range( n_trials ):

	# TIME FOR TRAINING #
	tic = tm.perf_counter() # t_start
	w, b = my_fit(Z_trn[:, :-1], Z_trn[:,-1])
 	# The my_fit function is called with the training data (Z_trn[:, :-1]
	# - all rows, except the last column, and Z_trn[:,-1] - the last column).
	# The returned weights w and bias b are assigned.
	toc = tm.perf_counter() # t_end
	t_train += toc - tic   # t_train = time of training = t_end - t_Start

	# number of features or dimensions in the trained model.
	d_size += w.shape[0]

	# TIME FOR MAPPING #
	tic = tm.perf_counter()
	# The my_map function is called with the testing data
	# (Z_tst[:, :-1]), and the resulting features are assigned to feat.
	feat = my_map( Z_tst[:, :-1] )
	toc = tm.perf_counter()
	t_map += toc - tic

	scores = feat.dot( w ) + b
	pred = np.zeros_like( scores )
	pred[scores > 0] = 1
	acc += np.average( Z_tst[ :, -1 ] == pred )

/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [ ]:
d_size /= n_trials
t_train /= n_trials
t_map /= n_trials
acc /= n_trials

print( d_size, t_train, t_map, 1 - acc )

561.0 11.547750719799978 0.04795899420000751 0.008500000000000063


In [ ]:
acc

0.9914999999999999